Install required packages

In [1]:
!pip install sentencepiece -q
!pip install demoji -q
!pip install torchmetrics[text] -q
!pip install transformers>=4.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 700.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 9.7 MB/s eta 0:00:00


Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import numpy as np
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, vocab
from collections import Counter, OrderedDict
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
from torch.hub import load_state_dict_from_url
import math
from typing import Iterable, List
from timeit import default_timer as timer
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url
import torchtext.data.functional as TDF
import sentencepiece as sp
from gensim.models import Word2Vec
import re
import demoji

Data loading and preprocessing

In [4]:
#load samsum dataset
def load_dataset(dataset, split):
  with open("drive/MyDrive/"+dataset+"/"+split+".json", "r") as f:
    data = json.load(f)
    return data

#preprocess to remove trailing carriage-return and newline
def preprocess(dataset):
  processed = []
  for x in dataset:
    processed.append({'summary' : re.sub("<[a-z]+_[a-z]+>","",demoji.replace(x['summary'].replace("\r","").replace("\n"," "), '')), 'dialogue' : re.sub("<[a-z]+_[a-z]+>","",demoji.replace(x['dialogue'].replace("\r","").replace("\n"," "), ''))})
  return processed

train = preprocess(load_dataset("corpus","train"))
val = preprocess(load_dataset("corpus","val"))
test = preprocess(load_dataset("corpus","test"))

Train sentenpiece tokenizer. 30k tokens. save model and load later

In [5]:
sp.SentencePieceTrainer.train(input='drive/MyDrive/sp_data.text', model_prefix='drive/MyDrive/sum_sp', vocab_size=30000, user_defined_symbols=['<pad>'])

Load tokenizer

In [6]:
spp = sp.SentencePieceProcessor()
spp.load("drive/MyDrive/sum_sp.model")

True

Build vocabulary

In [7]:
def make_vocab(fname):
  with open(fname, 'r') as f:
    lines=f.readlines()
  lines=[x.strip("\n").split("\t") for x in lines]
  tok2ids={}
  for i,x in enumerate(lines):
    tok2ids[x[0]]=i
  voc = vocab(OrderedDict(tok2ids), specials=['<unk>'])
  voc.set_default_index(voc['<unk>'])
  return voc
voc = make_vocab('drive/MyDrive/sum_sp.vocab')

Apply text transforms - tokenization, max len padding, bos & eos tokens affixed

In [8]:
padding_idx = 3
bos_idx = 1
eos_idx = 2
src_max_seq_len = 256
tgt_max_seq_len = 64

src_text_transform = T.Sequential(
    T.SentencePieceTokenizer('drive/MyDrive/sum_sp.model'),
    T.VocabTransform(voc),
    T.Truncate(src_max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
    T.ToTensor(padding_value=padding_idx),
    T.PadTransform(max_length=src_max_seq_len, pad_value=padding_idx),
)

tgt_text_transform = T.Sequential(
    T.SentencePieceTokenizer('drive/MyDrive/sum_sp.model'),
    T.VocabTransform(voc),
    T.Truncate(tgt_max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
    T.ToTensor(padding_value=padding_idx),
    T.PadTransform(max_length=tgt_max_seq_len, pad_value=padding_idx),
)

text_transform = T.Sequential(
    T.SentencePieceTokenizer('drive/MyDrive/sum_sp.model'),
    T.VocabTransform(voc),
    T.Truncate(src_max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

train word2vec embeddings from text and save model to load later

In [12]:
with open("drive/MyDrive/sp_data.text", 'r') as f:
    lines=f.readlines()
lines=[text_transform(x) for x in lines]
# lines = list(map(lambda x: x[0], lines))
w2v_model = Word2Vec(lines, min_count=1, window=7, workers=4, vector_size=256, sg=1)
w2v_model.save("drive/MyDrive/sp_word2vec.model")

export embeddings lookup-table to use later

In [ ]:

w2v_model.wv.key_to_index

In [ ]:
w2v_model

In [13]:
import numpy as np
dim = w2v_model.vector_size
vocab_size = len(spp)
table = np.array([w2v_model.wv[w2v_model.wv.key_to_index[i]] if i in w2v_model.wv.key_to_index.keys() else np.zeros([dim]) for i in range(vocab_size)])
# np.where(np.sum(table, axis=1)==0)[0].shape
np.save("drive/MyDrive/sp_emb.npy", table)

Load pre-trained embeddings and the tokenizer

In [6]:
def load_models(dir):
  spp = sp.SentencePieceProcessor()
  spp.load(dir+"sum_sp.model")
  embeddings = np.load(dir+"sp_emb.npy")
  return spp, embeddings

spp, embeddings = load_models("drive/MyDrive/")

Define the transformer network with  a positional embedding and token embedding layer and a linear generator to generate sequences

In [9]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
#
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int =256):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size) # denominator.
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        # print(token_embedding)
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        layer_norm = nn.LayerNorm(emb_size)
        encoder_layers = nn.TransformerEncoderLayer(emb_size, nhead, dim_feedforward, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers, layer_norm)
        decoder_layers = nn.TransformerDecoderLayer(emb_size, nhead, dim_feedforward, dropout)
        self.decoder = nn.TransformerDecoder(decoder_layers, num_decoder_layers, layer_norm)
        self.generator = nn.Linear(emb_size, vocab_size)
        self.tok_emb = TokenEmbedding(vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.tok_emb(src))
        tgt_emb = self.positional_encoding(self.tok_emb(trg))
        enc_out = self.encoder(src_emb, src_mask, src_padding_mask)
        dec_out = self.decoder(tgt_emb, enc_out, tgt_mask, None, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(dec_out)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.encoder(self.positional_encoding(
                            self.tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.decoder(self.positional_encoding(
                          self.tok_emb(tgt)), memory,
                          tgt_mask)

Utility functions to generate masks for pad tokens and masks for self attention in the decoder.

In [10]:
# function to create the target mask, to avoid peeking at subsequent tokens in the decoder self-attention stage.
# sz is the maximum length sequence in the batch.
# future token indices are filled with -inf and past token indices with 0. Binarized through exponentiation.
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# function returns source mask, target mask, source_padding_mask and target_padding_mask
# src - dialogue/text
# tgt - summary
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == padding_idx).transpose(0, 1)
    tgt_padding_mask = (tgt == padding_idx).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Define model parameters, hyperparameters, loss function, optimizer and a collate function for the data collator

In [11]:
# reproducibility seed
torch.manual_seed(0)
# tokens in vocabulary
VOCAB_SIZE = len(text_transform[1].vocab)
# token embedding dimension
EMB_SIZE = 256
# multi-heads in transformer
NHEAD = 4
# hidden forward network input dims
FFN_HID_DIM = 2048
# batch size
BATCH_SIZE = 32
# encoder and decoder layers in transformer
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

# seq2seq model init
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, VOCAB_SIZE, FFN_HID_DIM)

# model parameter init
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

with torch.no_grad():
  transformer.tok_emb.embedding.weight.copy_(torch.from_numpy(np.load("drive/MyDrive/sp_emb.npy")).float())

transformer = transformer.to(DEVICE)

# loss function
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=padding_idx)

#optimizer
optimizer = torch.optim.Adam(transformer.parameters())

# function to collate data samples into batch tensors
def collate_fn(batch):
    dialogues = [x['dialogue'] for x in batch]
    summaries = [x['summary'] for x in batch]
    return src_text_transform(dialogues).transpose(0,1), tgt_text_transform(summaries).transpose(0,1)

Train and Eval logic

In [12]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        # print(src.shape,tgt.shape)
        tgt_input = tgt[:-1, :]
        # print(tgt_input.shape)
        # break
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # print(src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
        # break

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        # tgt_out = tgt
        # print(logits.shape, tgt_out.shape)
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        # print(logits)
        # print(loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0
    val_dataloader = DataLoader(val, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        # tgt_out = tgt
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Utility functions for inference. 

In [13]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    # print(memory.shape)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        # print(ys.shape)
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        # print(ys.shape, memory.shape, tgt_mask.shape)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == eos_idx:
            break
        # print(prob.shape)
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = src_text_transform(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    # print(src_mask)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens, start_symbol=bos_idx).flatten()
    # print(tgt_tokens)
    return spp.decode(tgt_tokens.cpu().tolist())
    # return " ".join(text_transform[1].vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Training loop

In [14]:
NUM_EPOCHS = 10
min_loss = 1000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    if not epoch % 5:
      print(test[70])
      print(translate(transformer, test[70]['dialogue']))
    if val_loss < min_loss:
      min_loss = val_loss
      torch.save({
            'epoch': epoch,
            'model_state_dict': transformer.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss
            }, 'drive/MyDrive/checkpoints/pretrained_sp_tok_latest.pth')
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_jit_internal.py:1297: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn(


Epoch: 1, Train loss: 5.580, Val loss: 5.032, Epoch time = 62.473s
Epoch: 2, Train loss: 4.658, Val loss: 4.918, Epoch time = 60.336s
Epoch: 3, Train loss: 4.308, Val loss: 4.906, Epoch time = 63.040s
Epoch: 4, Train loss: 4.023, Val loss: 4.991, Epoch time = 61.690s
{'summary': "Ali left his wallet at Mohammad's place. Mohammad'll bring it to uni tomorrow.", 'dialogue': "Ali: I think I left my wallet at your place yesterday. Could you check?  Mohammad: Give me a sec, I'll have a look around my room. Ali: OK. Mohammad: Found it! Ali: Phew, I don't know what I'd do if it wasn't there. Can you bring it to uni tomorrow? Mohammad: Sure thing."}
The team is in the office is in the office.
Epoch: 5, Train loss: 3.774, Val loss: 5.112, Epoch time = 63.612s
Epoch: 6, Train loss: 3.559, Val loss: 5.245, Epoch time = 62.120s
Epoch: 7, Train loss: 3.368, Val loss: 5.299, Epoch time = 63.877s
Epoch: 8, Train loss: 3.189, Val loss: 5.391, Epoch time = 62.616s
Epoch: 9, Train loss: 3.023, Val loss: 

Import metrics

In [15]:
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bert import BERTScore
from torchmetrics.text import BLEUScore

Compute metrics on test data

In [26]:
targets = [x['summary'] for x in test]
preds = [translate(transformer, x['dialogue']) for x in test]
# print([preds])
# print([targets])
rouge_score=ROUGEScore()
bert_score = BERTScore()
bleu_score = BLEUScore()
# bleu = bleu_score(preds, targets)
rouge = rouge_score(preds,targets)
bert = bert_score(preds, targets)

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:42: UserWarning: The argument `model_name_or_path` was not specified while it is required when the default `transformers` model is used. It will use the default recommended model - 'roberta-large'.
  warnings.warn(*args, **kwargs)  # noqa: B028
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Display metrics

In [27]:
bleu_targets = [[x] for x in targets]
bleu = bleu_score(preds, bleu_targets)
print(bleu)

tensor(0.0046)


In [28]:
print(rouge)

{'rouge1_fmeasure': tensor(0.1242), 'rouge1_precision': tensor(0.1438), 'rouge1_recall': tensor(0.1212), 'rouge2_fmeasure': tensor(0.0102), 'rouge2_precision': tensor(0.0123), 'rouge2_recall': tensor(0.0097), 'rougeL_fmeasure': tensor(0.1029), 'rougeL_precision': tensor(0.1191), 'rougeL_recall': tensor(0.1009), 'rougeLsum_fmeasure': tensor(0.1162), 'rougeLsum_precision': tensor(0.1346), 'rougeLsum_recall': tensor(0.1135)}


In [29]:
bert = {k:v.mean() for k,v in bert.items()}
bert

{'precision': tensor(0.9767), 'recall': tensor(0.9712), 'f1': tensor(0.9739)}